In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from pandas_profiling import ProfileReport
import warnings
from geopy.geocoders import ArcGIS, Bing, Nominatim, OpenCage, GoogleV3, OpenMapQuest
import csv, sys
from geopy.geocoders import Nominatim 
import multiprocessing
import random as rd
from geopy.extra.rate_limiter import RateLimiter
from time import sleep
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import logging
import requests
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, wordpunct_tokenize, pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.corpus import stopwords

warnings.filterwarnings('ignore')


In [2]:
file_path=os.path.join("..","..","data","nlp-disaster","train.csv")
data=pd.read_csv(file_path)
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [27]:
data[data["target"]==1].head()

,id,keyword,location,text,target,new_location
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Italia
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Italia
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,Italia
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,Italia
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Italia


In [28]:
data[data["target"]==0].head()

,id,keyword,location,text,target,new_location
15,23,NaN,NaN,What's up man?,0,Italia
16,24,NaN,NaN,I love fruits,0,Italia
17,25,NaN,NaN,Summer is lovely,0,Italia
18,26,NaN,NaN,My car is so fast,0,Italia
19,28,NaN,NaN,What a goooooooaaaaaal!!!!!!,0,Italia


In [7]:
profile = ProfileReport(data, title='Profiling Report', explorative=True)
profile.to_file("static/eda.html")

In [45]:
arcgis = ArcGIS(timeout=50)
nominatim = Nominatim(timeout=50,user_agent="bogchalaca")
googlev3 = GoogleV3(timeout=50)

# choose and order your preference for geocoders here
geocoders = [nominatim]
def geocode(address):
    i = 0
    try:
        while i < len(geocoders):
            # try to geocode using a service
            location = geocoders[i].geocode(address)
            # if it returns a location
            if location != None:
                
                country=location[0].split(",")[-1]
                return country
            else:
                i += 1
    except:
        return np.nan

pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
#addresses = pool.map(geocode_worker, data["location"])
result=[]
for _ in tqdm(pool.imap(geocode, data["location"],chunksize=1), total=len(data["location"])):
    result.append(_)
    pass

100%|██████████| 7613/7613 [1:14:21<00:00,  1.71it/s]


In [58]:
import pickle
with open('location.pkl', 'wb') as f:
    pickle.dump(result, f)

In [4]:
import pickle

with open("data/location.pkl","rb") as f:
    result=pickle.load(f)

In [5]:
new_result=pd.DataFrame(result)
new_result.replace("Italia","None",inplace=True)

In [38]:
data["new_location"]=new_result
data_sample=data[data["keyword"].notna()].reset_index(drop=True)
final_data=pd.concat([data_sample,pd.get_dummies(data_sample["new_location"])],axis=1)
final_data=pd.concat([data_sample,pd.get_dummies(data_sample["keyword"],prefix="keyword")],axis=1)
y=final_data["target"]
text=final_data["text"]
final_data.drop(["keyword","new_location","id","location","target","text"],axis=1,inplace=True)

In [25]:
y.shape

(7552,)

In [39]:
stop_words = stopwords.words('english')

def clean_data(quote):
    quote = quote.lower()
    tokens = word_tokenize(quote)
    lemmatizer = WordNetLemmatizer()
    lem_words = [lemmatizer.lemmatize(w) for w in tokens]
    token_punc = [t for t in lem_words if t.isalpha()]
    token_stop = [t for t in token_punc if t not in stop_words]
    return " ".join(token_stop)


text = pd.DataFrame(text.apply(lambda x: clean_data(x)))

In [28]:
vectorizer = TfidfVectorizer(stop_words='english',min_df=0.01)
tf_idf = vectorizer.fit_transform(text["text"]).toarray()
tf_idf=pd.DataFrame(tf_idf,columns=vectorizer.get_feature_names())
final_data=pd.concat([final_data,tf_idf],axis=1)

In [42]:
final_data.head()

,keyword_ablaze,keyword_accident,keyword_aftershock,keyword_airplane%20accident,keyword_ambulance,keyword_annihilated,keyword_annihilation,keyword_apocalypse,keyword_armageddon,keyword_army,...,keyword_weapons,keyword_whirlwind,keyword_wild%20fires,keyword_wildfire,keyword_windstorm,keyword_wounded,keyword_wounds,keyword_wreck,keyword_wreckage,keyword_wrecked
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
with open('data/cleaned_text.pkl', 'wb') as f:
    pickle.dump(text, f)

In [9]:
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

final_data["polarity"] = data_sample["text"].apply(pol)
final_data["subjectivity"] = data_sample["text"].apply(sub)

In [43]:
final_data.shape

(7552, 221)

In [9]:
import pickle
with open('final_data.pkl', 'wb') as f:
    pickle.dump(final_data, f)

In [44]:
with open("data/final_data.pkl","rb") as f:
    final_data=pickle.load(f)

In [50]:
temp=final_data.iloc[10,:-2]
rfc.predict(np.array(temp).reshape(1,-1))

array([0])

In [16]:
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train,X_test,y_train,y_test=tts(final_data,y,train_size=0.8)
rfc=RandomForestClassifier(n_estimators=10,max_depth=30)
rfc.fit(X_train,y_train)
ypred=rfc.predict(X_test)
accuracy_score(y_test,ypred)


0.7246856386499008

In [17]:
rfc= pickle.load(open("models/rfc.sav","rb"))

In [19]:
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 30,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [17]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier(learning_rate=0.5)
lgb.fit(X_train,y_train)
ypred=lgb.predict(X_test)
accuracy_score(y_test,ypred)


0.7412309728656519

In [18]:
pickle.dump(rfc, open("rfc.sav", 'wb'))
pickle.dump(lgb, open("lgb.sav", 'wb'))

In [19]:
import tensorflow as tf

# Define a function
def model(input_dim):
    # Create the Sequential model
    model = tf.keras.models.Sequential()
  
    
    # Use a relu activation function
    model.add(tf.keras.layers.Dense(50, input_dim=input_dim, activation='relu'))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(100, activation='relu'))

    # Final layer is sigmoid for binary classification
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # return the model
    return model  

my_model=model(X_train.shape[1])

In [20]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

my_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=20),
            TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)]
my_model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=20, batch_size=16,callbacks=callbacks)
ypred = (my_model.predict(X_test)>=0.5)*1
accuracy_score(y_test,ypred)

Epoch 1/20
378/378 [==============================] - 2s 3ms/step - loss: 0.6251 - accuracy: 0.6313 - val_loss: 0.5222 - val_accuracy: 0.7478
Epoch 2/20
378/378 [==============================] - 1s 2ms/step - loss: 0.4574 - accuracy: 0.7935 - val_loss: 0.5153 - val_accuracy: 0.7591
Epoch 3/20
378/378 [==============================] - 1s 3ms/step - loss: 0.4117 - accuracy: 0.8178 - val_loss: 0.5278 - val_accuracy: 0.7498
Epoch 4/20
378/378 [==============================] - 1s 3ms/step - loss: 0.4098 - accuracy: 0.8159 - val_loss: 0.5517 - val_accuracy: 0.7472
Epoch 5/20
378/378 [==============================] - 1s 3ms/step - loss: 0.3678 - accuracy: 0.8354 - val_loss: 0.5471 - val_accuracy: 0.7518
Epoch 6/20
378/378 [==============================] - 1s 3ms/step - loss: 0.3575 - accuracy: 0.8414 - val_loss: 0.6043 - val_accuracy: 0.7432
Epoch 7/20
378/378 [==============================] - 1s 3ms/step - loss: 0.3333 - accuracy: 0.8512 - val_loss: 0.6270 - val_accuracy: 0.7459
Epoch 

0.7412309728656519

In [22]:
my_model.save('models/tnn')

INFO:tensorflow:Assets written to: models/tnn/assets


In [23]:
from transformers import DistilBertTokenizer
from transformers import BertModel
import torch

tokenizer = DistilBertTokenizer.from_pretrained("bert-base-uncased")


def process_comments(tokenizer, comments, max_length):
    input_ids, attention_mask = [], []
    for comment in tqdm(comments):
        proccessed_comment = tokenizer.encode_plus(comment, max_length=max_length, pad_to_max_length=True)
        input_ids.append(proccessed_comment["input_ids"])
        attention_mask.append(proccessed_comment["attention_mask"])
    return input_ids, attention_mask

max_length=50
input_ids, attention_mask = process_comments(tokenizer, data_sample["text"], max_length)
input_ids=torch.tensor(input_ids)
attention_mask=torch.tensor(attention_mask)
y = torch.tensor(y,dtype=torch.float32)
y=torch.reshape(y,(-1,))

100%|██████████| 7552/7552 [00:07<00:00, 985.14it/s] 


In [24]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset,DataLoader

(train_inputs,test_inputs, 
 train_mask, test_mask, 
 train_targets, test_targets) = train_test_split(input_ids, attention_mask, y, test_size=0.5)
train_data = TensorDataset(train_inputs, train_mask, train_targets)
test_data = TensorDataset(test_inputs, test_mask, test_targets)

In [25]:
from transformers import DistilBertForSequenceClassification
from torch.utils.data import Dataset, TensorDataset,DataLoader
from tqdm import tqdm

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=1)
train_loader = DataLoader(train_data,batch_size=8, shuffle=True)

NUM_EPOCHS = 1
LEARNING_RATE = 0.01
optimizer =torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.BCEWithLogitsLoss()
for i in range(NUM_EPOCHS):
    model.train()
    count=0
    for X_batch,X_attention_batch,y_batch in train_loader:
        print(count)
        output =   model(X_batch,attention_mask=X_attention_batch,labels=None)
        y_pred = output[0]
        y_pred = torch.reshape(y_pred,(-1,))
        loss = loss_fn(y_pred,y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        count+=8

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

0
8
16
24
32
40
48
56
64
72
80
88
96
104
112
120
128
136
144
152
160
168
176
184
192
200
208
216
224
232
240
248
256
264
272
280
288
296
304
312
320
328
336
344
352
360
368
376
384
392
400
408
416
424
432
440
448
456
464
472
480
488
496
504
512
520
528
536
544
552
560
568
576
584
592
600
608
616
624
632
640
648
656
664
672
680
688
696
704
712
720
728
736
744
752
760
768
776
784
792
800
808
816
824
832
840
848
856
864
872
880
888
896
904
912
920
928
936
944
952
960
968
976
984
992
1000
1008
1016
1024
1032
1040
1048
1056
1064
1072
1080
1088
1096
1104
1112
1120
1128
1136
1144
1152
1160
1168
1176
1184
1192
1200
1208
1216
1224
1232
1240
1248
1256
1264
1272
1280
1288
1296
1304
1312
1320
1328
1336
1344
1352
1360
1368
1376
1384
1392
1400
1408
1416
1424
1432
1440
1448
1456
1464
1472
1480
1488
1496
1504
1512
1520
1528
1536
1544
1552
1560
1568
1576
1584
1592
1600
1608
1616
1624
1632
1640
1648
1656
1664
1672
1680
1688
1696
1704
1712
1720
1728
1736
1744
1752
1760
1768
1776
1784
1792
1800
1808
1816


In [41]:
from sklearn import metrics

test_loader = DataLoader(test_data, batch_size=8, shuffle=False)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


preds = np.zeros([len(test_data), 1])
model.eval()
for i, (x_batch, x_mask) in enumerate(test_loader):
    outputs = model(x_batch,attention_mask=x_mask)
    y_pred = sigmoid(outputs[0].detach().numpy())
    preds[i*8:(i+1)*8, :] = y_pred
    
print(metrics.roc_auc_score(test_targets, preds))

0.8857054377405722


In [42]:
torch.save(model.state_dict(), "models/bert")